# Computing Partials and Amplitudes 

In [ ]:
import librosa
import librosa.display
import numpy as np
import os
from IPython import display as ipd
import matplotlib.pyplot as plt
import scipy
os.path.join('..')
from libsoni.core.methods import generate_tone_additive_synthesis
from libsoni.util.utils import normalize_signal

from scipy.io import wavfile

%matplotlib inline

In [ ]:
def pitch_to_frequency(pitch):
    return  440.0  * 2 ** ((pitch - 69) / 12)
def get_pitch_from_samplename(sample):
    return int(sample[7:10])
def get_folder_from_samplename(sample):
    return sample[:2]

## Computing Partials

In [ ]:
# loading the audio sample
Sample = '11Flu1F080m_np___0.wav'
SAMPLE_DIR = os.path.join('..','SNDB',get_folder_from_samplename(Sample), Sample)


pitch_sample = get_pitch_from_samplename(Sample)
f_sample = pitch_to_frequency(pitch_sample)

x, Fs = librosa.load(SAMPLE_DIR)
print('loaded sample:')
ipd.display(ipd.Audio(x, rate=Fs))

## Performing FFT

In [ ]:
# Define Length of FFT
N_fft = 16834

# Perform FFT to obtain the frequency domain representation
X = np.fft.fft(x, n = N_fft)

# Calculate the magnitudes of the FFT coefficients
magnitude = np.abs(X)

# Find the frequencies corresponding to each FFT coefficient
frequencies = np.fft.fftfreq(len(magnitude), 1 / Fs)

# Keep only the positive frequencies
frequencies = frequencies[:len(frequencies) // 2]
magnitude = magnitude[:len(magnitude) // 2]

plt.figure(figsize=(8, 4))
plt.plot(frequencies,magnitude)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('FFT for %s' %Sample )
#plt.legend()
plt.grid(True)
plt.show()
plt.show()

In [ ]:
# Define number of peaks to encounter for partials
N = 100

# Find N peaks in FFT
ind = np.argpartition(magnitude, -N)[-N:]

# Get parameters for synthesis
frequencies_synthesis = frequencies[ind]
magnitudes_synthesis = magnitude[ind]

partials = frequencies_synthesis / f_sample
partials_amplitude = magnitudes_synthesis

# perform synthesis
resyn_1 = generate_tone_additive_synthesis(pitch=pitch_sample,
                                       partials=partials,
                                       partials_amplitudes=partials_amplitude,
                                       duration_sec = 3.0)

resyn_1 = normalize_signal(resyn_1)

print('Re-synthesized sample using %s peaks of fft:' %N)
ipd.display(ipd.Audio(resyn_1, rate=Fs))

In [ ]:
for N in [1,5,10,50,100,200,500]:
    # Find N peaks in FFT
    ind = np.argpartition(magnitude, -N)[-N:]
    
    # Get parameters for synthesis
    frequencies_synthesis = frequencies[ind]
    magnitudes_synthesis = magnitude[ind]
    
    partials = frequencies_synthesis / f_sample
    partials_amplitude = magnitudes_synthesis
    
    # perform synthesis
    resyn_N = generate_tone_additive_synthesis(pitch=pitch_sample,
                                           partials=partials,
                                           partials_amplitudes=partials_amplitude,
                                           duration_sec = 3.0)
    
    resyn_N = normalize_signal(resyn_N)
    
    print('Re-synthesized Sample using %s peaks of fft:' %N)
    ipd.display(ipd.Audio(resyn_N, rate=Fs))

## Different Approach: Search Peaks in Octave-Subbands

In [ ]:
# Define the number of octave bands
num_octave_bands = 8
num_peaks_per_octave_band = 1

# Calculate the center frequencies of the octave bands
center_frequencies = [f_sample * i for i in range(1, num_octave_bands)]


plt.figure(figsize=(8, 4))

partials = []
partials_amplitude = []

for center_freq in center_frequencies:

    lower_freq = center_freq - f_sample / 2
    upper_freq = center_freq + f_sample / 2
        
    band_indices = np.where((frequencies >= lower_freq) & (frequencies < upper_freq))
    
    frequencies_band = frequencies[band_indices]
    magnitude_band = magnitude[band_indices]
    
    
    plt.axvline(lower_freq, color='r', linestyle='--', alpha=0.5)
    plt.axvline(upper_freq, color='r', linestyle='--', alpha=0.5)
    plt.plot(frequencies[band_indices],magnitude[band_indices])


    
    partials.append(frequencies_band[np.argpartition(magnitude_band, -num_peaks_per_octave_band)[-num_peaks_per_octave_band:]][0]/f_sample)
    partials_amplitude.append(magnitude_band[np.argpartition(magnitude_band, -num_peaks_per_octave_band)[-num_peaks_per_octave_band:]][0])

    

resyn_2 = generate_tone_additive_synthesis(pitch=pitch_sample,
                                           partials=partials,
                                           partials_amplitudes=partials_amplitude,
                                           duration_sec = 3.0)

resyn_2 = normalize_signal(resyn_2)

ipd.display(ipd.Audio(resyn_2, rate=Fs))
plt.plot()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Sub-Octave Bands')
plt.grid(True)
plt.show()

In [ ]:
stft = librosa.stft(x)



In [ ]:
# Plot the original waveform, the STFT, and the envelope
plt.figure(figsize=(12, 8))

# Original waveform
plt.subplot(3, 1, 1)
librosa.display.waveshow(audio_data, sr=sample_rate)
plt.title('Original Audio Waveform')

# STFT
plt.subplot(3, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(np.abs(stft), ref=np.max), y_axis='log', x_axis='time', sr=sample_rate)
plt.colorbar(format='%+2.0f dB')
plt.title('STFT Magnitude Spectrogram')



In [ ]:
print(np.abs(stft).shape)
print(len(magnitude))